### STEP : PICRUST2 Analysis



#### Example

- [PICRUST2 tutorial](https://github.com/picrust/picrust2/wiki/q2-picrust2-Tutorial)
- [Limitations](https://github.com/picrust/picrust2/wiki/Key-Limitations)


#### Methods
- [composition](https://docs.qiime2.org/2022.8/plugins/available/composition/)

## Setup and settings

In [1]:
# Importing packages
import os
import biom
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.feature_table.visualizers import summarize

from picrust2.pipeline import full_pipeline
from picrust2.default import (default_ref_dir, default_tables, default_regroup_map, default_pathway_map)
from qiime2.plugins import picrust2

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [3]:
metadata_file = '/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [4]:
# Parameters
experiment_name = "miliane-CxAC-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/manifest/miliane-manifest-CxAC.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/dados_brutos_rede_genoma/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [5]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [6]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reqs_path = os.path.join(qiime_folder, 'dada2-reps.qza')

# PICRUST@ folder
picrust2_folder = os.path.abspath(os.path.join(experiment_folder, 'picrust2'))

# Create path if it not exist
if not os.path.isdir(picrust2_folder):
    os.makedirs(picrust2_folder)
    print(f'New picrust2-artifacts folder path created: {picrust2_folder}')

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [7]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)

#Load FeatureTable[Sequence]
seqs = Artifact.load(dada2_reqs_path)

### Execute full pipelie

The entire PICRUSt2 pipeline will be run using a single method, called `picrust2.methods.full_pipeline`. This method will run each of the 4 key steps: 

1. sequence placement
2. hidden-state prediction of genomes
3. metagenome prediction
4. pathway-level predictions.

More information on [Documentation](https://github.com/picrust/picrust2/wiki/Full-pipeline-script).

In [8]:

# Execute full pipeline
results = picrust2.methods.full_pipeline(
    table=tabs, 
    seq=seqs, 
    threads=6, 
    placement_tool='sepp',
    hsp_method='pic', 
    max_nsti=2,
    highly_verbose=True
)

918 of 918 sequence ids overlap between input table and FASTA.

Placing sequences onto reference tree
place_seqs.py --study_fasta /tmp/tmp4pj1gzlt/seqs.fna --ref_dir /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref --out_tree /tmp/tmp4pj1gzlt/picrust2_out/out.tre --processes 6 --intermediate /tmp/tmp4pj1gzlt/picrust2_out/intermediate/place_seqs --min_align 0.8 --chunk_size 5000 --placement_tool sepp --verbose



['run_sepp.py', '--tree', '/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.tre', '--raxml', '/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.raxml_info', '--cpu', '6', '--molecule', 'dna', '--outdir', '/tmp/tmp4pj1gzlt/picrust2_out/intermediate/place_seqs/sepp_out', '-seed', '297834', '--alignment', '/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.fna', '--fragment', '/tmp/tmp4pj1gzlt/picrust2_out/intermediate/place_seqs/study_seqs_filtered.fasta']

                                              ....      ....  
                                             '' '||.   .||'   
                                                  ||  ||      
                                                  '|.|'       
     ...'   ....   ... ...  ... ...   ....        .|'|.       
    |  ||  

hmmalign --trim --dna --mapali /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.fna --informat FASTA -o /tmp/tmp4pj1gzlt/picrust2_out/intermediate/place_seqs/query_align.stockholm /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.hmm /tmp/tmp4pj1gzlt/seqs.fna

Warning - 2 input sequences aligned poorly to reference sequences (--min_align option specified a minimum proportion of 0.8 aligning to reference sequences). These input sequences will not be placed and will be excluded from downstream steps.

This is the set of poorly aligned input sequences to be excluded: 6503004be0b011b96563e401dc4e5daa, 3e242896853a3bc57b1f8589ed17d0e7

Raw input sequences ranged in length from 271 to 430

run_sepp.py --tree /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/pro_ref/pro_ref.tre --raxml /home/lauro/

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_nsti.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpvmu30lpg/known_tips.txt /tmp/tmpvmu30lpg/nsti_out.txt

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpxt1vqfj3/subset_tab_0 pic 0.5 FALSE FALSE /tmp/tmp1m14a93w/predicted_counts.txt /tmp/tmp1m14a93w/predicted_ci.txt 100


hsp.py --tree /tmp/tmp4pj1gzlt/picrust2_out/out.tre --output /tmp/tmp4pj1gzlt/picrust2_out/EC_predicted.tsv.gz --observed_trait_table /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/prokaryotic/ec.txt.gz --hsp_method pic --edge_exponent 0.5 --seed 100 --processes 6 --verbose


Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpof82cjtu/subset_tab_4 pic 0.5 FALSE FALSE /tmp/tmpdps9kukb/predicted_counts.txt /tmp/tmpdps9kukb/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpof82cjtu/subset_tab_5 pic 0.5 FALSE FALSE /tmp/tmp19nm09k3/predicted_counts.txt /tmp/tmp19nm09k3/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpof82cjtu/subset_tab_3 pic 0.5 FALSE FALSE /tmp/tmpd59cgjnh/predicted_counts.txt /tmp/tmpd59cgjnh/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpof82cjtu/subset_tab_0 p

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpn2r6ibcx/subset_tab_1 pic 0.5 FALSE FALSE /tmp/tmpkg2o3mm4/predicted_counts.txt /tmp/tmpkg2o3mm4/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpn2r6ibcx/subset_tab_10 pic 0.5 FALSE FALSE /tmp/tmpe6tm7qb8/predicted_counts.txt /tmp/tmpe6tm7qb8/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpn2r6ibcx/subset_tab_17 pic 0.5 FALSE FALSE /tmp/tmpo3i7yga6/predicted_counts.txt /tmp/tmpo3i7yga6/predicted_ci.txt 100

Rscript /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/Rscripts/castor_hsp.R /tmp/tmp4pj1gzlt/picrust2_out/out.tre /tmp/tmpn2r6ibcx/subset_tab_2

All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.

Running metagenome pipeline for KO
metagenome_pipeline.py --input /tmp/tmp4pj1gzlt/intable.biom --function /tmp/tmp4pj1gzlt/picrust2_out/KO_predicted.tsv.gz --min_reads 1 --min_samples 1 --out_dir /tmp/tmp4pj1gzlt/picrust2_out/KO_metagenome_out --max_nsti 2 --marker /tmp/tmp4pj1gzlt/picrust2_out/marker_predicted_and_nsti.tsv.gz



Inferring pathways from predicted EC


All ASVs were below the max NSTI cut-off of 2.0 and so all were retained for downstream analyses.

pathway_pipeline.py --input /tmp/tmp4pj1gzlt/picrust2_out/EC_metagenome_out/pred_metagenome_unstrat.tsv.gz --out_dir /tmp/tmp4pj1gzlt/picrust2_out/pathways_out --map /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/pathway_mapfiles/metacyc_path2rxn_struc_filt_pro.txt --intermediate /tmp/tmp4pj1gzlt/picrust2_out/intermediate/pathways --proc 6 --regroup_map /home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/picrust2/default_files/pathway_mapfiles/ec_level4_to_metacyc_rxn.tsv --verbose

Wrote predicted pathway abundances and coverages to /tmp/tmp4pj1gzlt/picrust2_out/pathways_out


### Load created artifacts

We will define file paths and persist all artifacts as `qza` files. In sequence, we will persist the visualizations files as `qzv` files.

In [10]:
ec_path = os.path.join(picrust2_folder, 'ec-pred-metagen.qza')
results.ec_metagenome.export_data(output_dir=ec_path.split('.')[0])
results.ec_metagenome.save(ec_path)

ko_path = os.path.join(picrust2_folder, 'ko-pred-metagen.qza')
results.ko_metagenome.export_data(output_dir=ko_path.split('.')[0])
results.ko_metagenome.save(ko_path)

pathway_path = os.path.join(picrust2_folder, 'pathway-abundance.qza')
results.pathway_abundance.export_data(output_dir=pathway_path.split('.')[0])
results.pathway_abundance.save(pathway_path)

'/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/experiments/miliane-CxAC-trim/picrust2/pathway-abundance.qza'

In [11]:
ec_viz = summarize(table=results.ec_metagenome, sample_metadata=metadata_qa).visualization
ec_viz.save(ec_path[:-1]+'v')
ko_viz = summarize(table=results.ko_metagenome, sample_metadata=metadata_qa).visualization
ko_viz.save(ko_path[:-1]+'v')
path_viz = summarize(table=results.pathway_abundance, sample_metadata=metadata_qa).visualization
path_viz.save(pathway_path[:-1]+'v')

'/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/experiments/miliane-CxAC-trim/picrust2/pathway-abundance.qzv'

In [21]:
ec_fpath = os.path.join(picrust2_folder, 'ec.tsv')
ko_fpath = os.path.join(picrust2_folder, 'ko.tsv')
pathway_fpath = os.path.join(picrust2_folder, 'pathway.tsv')

ec_desc_fpath = os.path.join(picrust2_folder, 'ec-desc.tsv')
ko_desc_fpath = os.path.join(picrust2_folder, 'ko-desc.tsv')
pathway_desc_fpath = os.path.join(picrust2_folder, 'pathway-desc.tsv')


df = results.ec_metagenome.view(pd.DataFrame).T
# df = biom.Table.to_dataframe(res)
df.to_csv(ec_fpath, sep='\t', index=True)

df = results.ko_metagenome.view(pd.DataFrame).T
# df = biom.Table.to_dataframe(res)
df.to_csv(ko_fpath, sep='\t', index=True)

df = results.pathway_abundance.view(pd.DataFrame).T
# df = biom.Table.to_dataframe(res)
df.to_csv(pathway_fpath, sep='\t', index=True)

In [22]:
!add_descriptions.py -i {ec_fpath} -m EC -o {ec_desc_fpath}
!add_descriptions.py -i {ko_fpath} -m KO -o {ko_desc_fpath}
!add_descriptions.py -i {pathway_fpath} -m METACYC -o {pathway_desc_fpath}